In [36]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import numpy as np

from ICARUS.Core.units import calc_mach, calc_reynolds
from ICARUS.Database import DB
from ICARUS.Airfoils.airfoil import Airfoil
from ICARUS.Core.struct import Struct
from ICARUS.Computation.Solvers.solver import Solver
from ICARUS.Computation.Solvers.OpenFoam.files.setup_case import MeshType

# Reynolds And Mach and AoA

In [ ]:
# Assumptions
chord_max: float = 0.5
chord_min: float = 0.1
u_max: float = 30.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_min: float = calc_mach(10, speed_of_sound)
mach_max: float = calc_mach(30, speed_of_sound)
mach = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max

# REYNOLDS ESTIMATION
reynolds_max: float = calc_reynolds(u_max, chord_max, viscosity)
reynolds_min: float = calc_reynolds(u_min, chord_min, viscosity)
reynolds = np.logspace(
    start=np.log10(reynolds_min),
    stop=np.log10(reynolds_max),
    num=5,
    base=10,
)
REYN = reynolds_max

# ANGLE OF ATTACK SETUP
aoa_max: float = 2
aoa_min: float = -2
num_of_angles: float = (aoa_max - aoa_min) * 2 + 1
angles = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}

In [ ]:
cleaning = False
calcF2W = False
calcOpenFoam = False
calcXFoil = True

# Get Airfoil

In [ ]:
airfoil_names = list(DB.foils_db.airfoils.keys())

In [ ]:
airfoils = [airfoil  for airfoil in airfoil_names if 
            (
                airfoil.upper().startswith('S') or 
                airfoil.upper().startswith('AG') or 
                airfoil.upper().startswith('CLARK') or
                airfoil.upper().startswith('DAE') or
                airfoil.upper().startswith('E') or
                airfoil.upper().startswith('H') or
                airfoil.upper().startswith('M') or
                airfoil.upper().startswith('N') or
                airfoil.upper().startswith('O') or
                airfoil.upper().startswith('W') 
            )
]
print(len(airfoils))


# Foil2Wake

In [ ]:
if calcF2W :
    # Import Solver
    from ICARUS.Computation.Solvers.Foil2Wake.f2w_section import Foil2Wake
    f2w_s: Solver = Foil2Wake()

    # Import Analysis
    analysis: str = f2w_s.get_analyses_names()[0]  # Run
    f2w_s.select_analysis(analysis)

    # Get Options
    options: Struct = f2w_s.get_analysis_options(verbose=True)
    solver_parameters: Struct = f2w_s.get_solver_parameters()

    # Set Options
    options.airfoil = DB.foils_db.airfoils[airfoils[0]]
    options.reynolds = reynolds
    options.mach = MACH
    options.f_trip_low = ftrip_low
    options.f_trip_upper = ftrip_up
    options.angles = angles

    # RUN
    f2w_s.set_analysis_options(options)
    f2w_s.print_analysis_options()
    f2w_s.execute()

    # Get Results
    polars = f2w_s.get_results()


# Xfoil

In [35]:
if calcXFoil:
    from ICARUS.Computation.Solvers.Xfoil.xfoil import Xfoil 
    import matplotlib.pyplot as plt
    import time

    for airfoil in airfoils:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get airfoil
        airf: Airfoil = DB.foils_db.airfoils[airfoil]
        airf.selig = airf.to_selig()
        # airf.plot()
        xfoil: Solver = Xfoil()

        # Import Analysis
        analysis: str = xfoil.get_analyses_names()[1]  # Run
        xfoil.select_analysis(analysis)

        # Get Options
        options = xfoil.get_analysis_options(verbose=False)
        solver_parameters = xfoil.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.mach = MACH
        options.reynolds = reynolds
        options.min_aoa = -5
        options.max_aoa = 5
        options.aoa_step = 1

        # Set Solver Options
        solver_parameters.max_iter = 1000
        solver_parameters.Ncrit = 9
        solver_parameters.xtr = (0.1, 0.1)
        solver_parameters.print = True
        
        xfoil.define_analysis(options, solver_parameters)
        xfoil.print_analysis_options()
        
        # RUN
        xfoil.execute()

Analysis Completed

Running airfoil W1011

Airfoil Polar Analysis For a multiple Reynolds zeroing BL
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds zeroing BL: 

| VarName   | Value                    | Description                       |
|-----------|--------------------------|-----------------------------------|
| airfoil   | Complex Datatype (w1011) | Airfoil Object                    |
| mach      | Complex Datatype         | Mach number                       |
| reynolds  | Complex Datatype         | List of Reynold's numbers to run  |
| min_aoa   | -5                       | Minimum angle of attack           |
| max_aoa   | 5                        | Maximum angle of attack           |
| aoa_step  | 1                        | Step between each angle of attack |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver XFoil:
	Analysis Airfoil Polar Analysis For a multiple 

  0%|          | 0/5 [00:00<?, ?it/s]

Analysis Completed

Running airfoil W1015

Airfoil Polar Analysis For a multiple Reynolds zeroing BL
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds zeroing BL: 

| VarName   | Value                    | Description                       |
|-----------|--------------------------|-----------------------------------|
| airfoil   | Complex Datatype (w1015) | Airfoil Object                    |
| mach      | Complex Datatype         | Mach number                       |
| reynolds  | Complex Datatype         | List of Reynold's numbers to run  |
| min_aoa   | -5                       | Minimum angle of attack           |
| max_aoa   | 5                        | Maximum angle of attack           |
| aoa_step  | 1                        | Step between each angle of attack |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver XFoil:
	Analysis Airfoil Polar Analysis For a multiple 

  0%|          | 0/5 [00:00<?, ?it/s]

Analysis Completed

Running airfoil WASPSM

Airfoil Polar Analysis For a multiple Reynolds zeroing BL
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds zeroing BL: 

| VarName   | Value                     | Description                       |
|-----------|---------------------------|-----------------------------------|
| airfoil   | Complex Datatype (waspsm) | Airfoil Object                    |
| mach      | Complex Datatype          | Mach number                       |
| reynolds  | Complex Datatype          | List of Reynold's numbers to run  |
| min_aoa   | -5                        | Minimum angle of attack           |
| max_aoa   | 5                         | Maximum angle of attack           |
| aoa_step  | 1                         | Step between each angle of attack |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver XFoil:
	Analysis Airfoil Polar Analysis For a 

...

Analysis Completed

Running airfoil WB140

Airfoil Polar Analysis For a multiple Reynolds zeroing BL
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds zeroing BL: 

| VarName   | Value                    | Description                       |
|-----------|--------------------------|-----------------------------------|
| airfoil   | Complex Datatype (wb140) | Airfoil Object                    |
| mach      | Complex Datatype         | Mach number                       |
| reynolds  | Complex Datatype         | List of Reynold's numbers to run  |
| min_aoa   | -5                       | Minimum angle of attack           |
| max_aoa   | 5                        | Maximum angle of attack           |
| aoa_step  | 1                        | Step between each angle of attack |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver XFoil:
	Analysis Airfoil Polar Analysis For a multiple 

  0%|          | 0/5 [00:00<?, ?it/s]

Analysis Completed

Running airfoil WHITCOMB

Airfoil Polar Analysis For a multiple Reynolds zeroing BL
Available Options of Xfoil for Airfoil Polar Analysis For a multiple Reynolds zeroing BL: 

| VarName   | Value                       | Description                       |
|-----------|-----------------------------|-----------------------------------|
| airfoil   | Complex Datatype (whitcomb) | Airfoil Object                    |
| mach      | Complex Datatype            | Mach number                       |
| reynolds  | Complex Datatype            | List of Reynold's numbers to run  |
| min_aoa   | -5                          | Minimum angle of attack           |
| max_aoa   | 5                           | Maximum angle of attack           |
| aoa_step  | 1                           | Step between each angle of attack |

If there are Multiple Values, or complex datatypes, or N/A you should inspect them sepretly by calling the option name

Running Solver XFoil:
	Analysis Airfoil Pol

  0%|          | 0/5 [00:00<?, ?it/s]

Analysis Completed


In [ ]:
print(airfoils[1])
DB.foils_db.polars[airfoils[1]]

# Mapflow

# OpenFoam (NOT HIGH MACH)

In [ ]:
if calcOpenFoam:
    airfoils: list[str] = [airf.name]#["0008"]
    from ICARUS.Computation.Solvers.OpenFoam.open_foam import OpenFoam

    for airfoil in airfoils:
        print(f"\nRunning airfoil {airfoil}\n")
        # # Get Airfoil
        airf: Airfoil = Airfoil.naca(airfoil, n_points=200)
        # airf.plot()
        open_foam: Solver = OpenFoam()

        # Import Analysis
        analysis: str = open_foam.get_analyses_names()[0]  # Run
        open_foam.select_analysis(analysis)

        # Get Options
        options = open_foam.get_analysis_options(verbose=True)
        solver_parameters = open_foam.get_solver_parameters()

        # Set Options
        options.airfoil = airf
        options.angles = angles
        options.reynolds = REYN
        options.mach = MACH
        open_foam.print_analysis_options()

        # Set Solver Options
        solver_parameters.mesh_type = MeshType.structAirfoilMesher
        solver_parameters.max_iterations = 20
        solver_parameters.silent = False
        
        # RUN
        open_foam.set_analysis_options(options)
        open_foam.print_analysis_options()
        open_foam.execute()